In [1]:
from textblob import TextBlob
from textblob import Word
import re

In [7]:
from ugot import ugot
import random
import time

got = ugot.UGOT()

got.initialize('192.168.1.186')

192.168.1.186:50051


_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:192.168.1.186:50051: ConnectEx: Connection timed out (A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond.
 -- 10060)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:192.168.1.186:50051: ConnectEx: Connection timed out (A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond.\r\n -- 10060)", grpc_status:14, created_time:"2024-12-29T11:21:47.9294125+00:00"}"
>

In [ ]:
def error_check(text):
        # Create a TextBlob for the input
        blob = TextBlob(text)
        
        # Correct the input
        corrected_input = blob.correct()
        
        # Check if correction is needed
        if str(blob) != str(corrected_input):
            print(f"Did you mean: '{corrected_input}'?")
            confirmation = input("Please type 'yes' if correct, or 'no' to keep your original text: ").strip().lower()
            
            if confirmation == "yes":
                print("Fixing...")
                return corrected_input
            else:
                print("No problem, I'll keep your original input.")
                return text
        else:
            print("No errors detected...")
            return text

# Run the chatbot
error_check("forward 10")


In [ ]:
def validate_chained_commands(commands):
    # Define the regex pattern
    pattern = r"^(?:\s*(forward|backward|left|right) \d+\s*)+$"
    
    # Match the entire input string against the pattern
    if re.match(pattern, commands):
        print(f"'{commands}' is valid.")
        return True
    else:
        print(f"'{commands}' is not valid.")
        return False

def command_validator():
    print("Welcome to the Chained Command Validator!")
    print("Valid commands are: forward, backward, left, right, followed by a distance (e.g., 'forward 10').")
    print("You can chain multiple commands (e.g., 'forward 10 backward 5 left 20').")
    print("Type 'exit' to quit.")

    while True:
        # Get user input
        user_input = input("\nEnter your commands: ").strip().lower()
        if user_input == "exit":
            print("Goodbye!")
            break

        # Validate the chained commands
        validate_chained_commands(user_input)

# Run the command validator
command_validator()


In [ ]:
def ugot_regex():
    commanding = True
    
    command_list = ['forward', 'backward', 'left', 'right']
    
    print("Welcome to the UGOT Command Center! Type 'exit' to quit. Type 'help' for a list of commands.")
    
    while commanding:
        print("What are your orders?")
        command = input("> ")
        command = command.lower()
        
        command = str(error_check(command))
        
        if command == 'exit':
            commanding = False
            print("Exiting robot...")

        elif command == 'help':
            print("Here are the available commands:")
            for j in command_list:
                print(f"- {j}")
        
        elif validate_chained_commands(command):
            command = command.split(' ')
            for i, values in enumerate(command):
                if values == 'forward':
                    got.mecanum_move_speed_times(0, 20, int(command[i+1]), 1)

                if values == 'backward':
                    got.mecanum_move_speed_times(1, 20, int(command[i+1]), 1)

                if values == 'left':
                    got.mecanum_turn_speed_times(2, 30, int(command[i+1]), 2)

                if values == 'right':
                    got.mecanum_turn_speed_times(3, 30, int(command[i+1]), 2)
                    
        else:
            print("I do not understand that command")

ugot_regex()
        

In [ ]:
def calculator():
    print("Welcome to the Python Calculator!")
    print("You can perform the following operations:")
    print("Addition (+), Subtraction (-), Multiplication (*), Division (/)")
    print("Type 'exit' to quit.")

    # Dictionary of operations with lambda functions
    operations = {
        '+': lambda x, y: x + y,
        '-': lambda x, y: x - y,
        '*': lambda x, y: x * y,
        '/': lambda x, y: x / y if y != 0 else "Error: Division by zero is not allowed."
    }

    while True:
        # Get user input for the operation
        operation = input("\nEnter the operation (+, -, *, /) or 'exit' to quit: ").strip()

        if operation.lower() == 'exit':
            print("Goodbye!")
            break

        if operation not in operations:
            print("Invalid operation. Please choose +, -, *, or /.")
            continue

        try:
            # Get the numbers
            num1 = float(input("Enter the first number: "))
            num2 = float(input("Enter the second number: "))

            # Perform the calculation using the dictionary
            result = operations[operation](num1, num2)

            print(f"The result is: {result}")
        except ValueError:
            print("Invalid input. Please enter numeric value")


def guess_the_number():
    print("Welcome to the Guess the Number Game!")
    print("Choose a difficulty level:")
    print("1. Easy (Exact hints: Too high/Too low)")
    print("2. Medium (Relative range hints: Within 10, Within 5)")
    print("3. Hard (General feedback only: Hot/Cold)")

    # Select difficulty level
    while True:
        try:
            difficulty = int(input("Enter the number corresponding to the difficulty level (1/2/3): "))
            if difficulty in [1, 2, 3]:
                break
            else:
                print("Invalid choice. Please enter 1, 2, or 3.")
        except ValueError:
            print("Please enter a valid number.")

    # Generate a random number
    secret_number = random.randint(1, 100)
    attempts = 0

    print("\nI've picked a number between 1 and 100. Can you guess it?")
    
    while True:
        try:
            guess = int(input("Your guess: "))
            attempts += 1
            
            if guess == secret_number:
                print(f"Congratulations! You guessed it in {attempts} attempts.")
                break
            
            # Provide hints based on difficulty level
            if difficulty == 1:  # Easy: Exact hints
                if guess < secret_number:
                    print("Too low! Try again.")
                else:
                    print("Too high! Try again.")
            elif difficulty == 2:  # Medium: Relative range hints
                difference = abs(secret_number - guess)
                if difference <= 5:
                    print("Very close! You're within 5.")
                elif difference <= 10:
                    print("Close! You're within 10.")
                else:
                    print("You're far off. Try again.")
            elif difficulty == 3:  # Hard: General feedback
                difference = abs(secret_number - guess)
                if difference <= 5:
                    print("Hot!")
                elif difference <= 15:
                    print("Warm.")
                else:
                    print("Cold.")
        except ValueError:
            print("Please enter a valid number.")

#Function for adding a user
def add_user(users):
    adding = True
    
    while adding:
        new_username = input("Add username here >")

        if new_username in users: 
            print("Error: user already exists!")
        else:
            new_password = input("Add password here >")
            users[new_username] = new_password
            adding = False

In [ ]:
def mood_chatbot_bayes():
    print("Hi! I'm your friendly chatbot :)")
    print("How are you feeling today?")
    
    # Get user input
    user_response = input("You: ")
    
    # Analyze sentiment using TextBlob
    blob = TextBlob(user_response, analyzer=NaiveBayesAnalyzer())
    sentiment = blob.sentiment.p_pos
    
    # Evaluate the sentiment and respond accordingly
    if sentiment > 0.5:
        print("Chatbot: That's great to hear! You seem really happy today!")
    elif sentiment > 0:
        print("Chatbot: Sounds like you're doing pretty well!")
    elif sentiment == 0:
        print("Chatbot: Hmm, I can't quite tell how you're feeling. Neutral, maybe?")
    else:
        print("Chatbot: Oh no! It seems like you're feeling a bit down. I'm here if you want to talk.")

    print("Chatbot: Thanks for sharing! Have a great day!")

In [ ]:
users = {'rafa': 'password'}



logged_in = False

while not logged_in:
    username = input("Please enter your username >")
    password = input("Please enter your password >")
    
    if username in users:
        if users[username] == password:
            print(f"Welcome, {username}!")
            logged_in = True
        else:
            print("Invalid username or password")
    else:
        print("Invalid username or password")
   



#Dictionary to hold all the commands
commands = {
    "exit": "Exits the chatbot",
    "help": "Displays the list of available commands",
    "add user": "Adds a new user to the chatbot",
    "calculator": "Uses the calculator",
    "game": "Play a simple number guessing game",
    "ugot": "Enters the UGOT command center",
    "mood": "Analyses your mood"
}

print("\nHi! I am a ChatBot. \nType \'help\' for a list of commands. \
\nType \'exit\' to exit the chatbot.'")

while True:
    print("What would you like to do?")
    prompt = input("You: ")
    low = prompt.lower()
    
    if 'exit' in low:
        print("See you!")
        break
        
    elif 'help' in low:
        print("\nChatBot: Here are the available commands:")
        for command, description in commands.items():
            print(f"- {command}: {description}")
            
    elif 'hello' in low:
        print("Hello! How are you doing?")
        input("You: ")
        print("I see!")
        
    elif "add user" in low:
        add_user(users)
        print("New user added sucessfully!")
        
    elif 'calc' in low:
        calculator()
        
    elif 'game' in low:
        guess_the_number()
    
    elif 'ugot' in low:
        ugot_regex()

    elif 'mood'in low:
        mood_chatbot_bayes()
        
    else:
        print("I don't understand that command.")
